# 🎓 Support Vector Machines


Neste aula você usará SVM (Support Vector Machines) para construir e treinar um modelo usando registros de células humanas e classificar as células se as amostras são benignas ou malignas.

O SVM funciona mapeando dados para um espaço de recursos de alta dimensão para que os pontos de dados possam ser categorizados, mesmo quando os dados não são linearmente separáveis. Um separador entre as categorias é encontrado, então os dados são transformados de tal forma que o separador pode ser desenhado como um hiperplano. Em seguida, as características dos novos dados podem ser usadas para prever o grupo ao qual um novo registro deve pertencer.

In [1]:
# Dependências
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Importação e limpeza dos dados

1) Importe os dados da seguinte url: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv

In [2]:
dados = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv')
dados

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


O campo ID contém os identificadores do paciente. As características das amostras de células de cada paciente estão contidas nos campos Clump à Mit. Os valores são classificados de 1 a 10, sendo 1 o mais próximo de benigno.

O campo Class contém o diagnóstico, conforme confirmado por procedimentos médicos separados, se as amostras são benignas (valor = 2) ou malignas (valor = 4).

Vejamos a distribuição das classes com base Clump thickness e Uniformity of cell size:

2) Use o método `info()` para chechar quais colunas possuem valroes não numéricos.

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           699 non-null    int64 
 1   Clump        699 non-null    int64 
 2   UnifSize     699 non-null    int64 
 3   UnifShape    699 non-null    int64 
 4   MargAdh      699 non-null    int64 
 5   SingEpiSize  699 non-null    int64 
 6   BareNuc      699 non-null    object
 7   BlandChrom   699 non-null    int64 
 8   NormNucl     699 non-null    int64 
 9   Mit          699 non-null    int64 
 10  Class        699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


Note que a coluna `BareNuc`possui valores não numéricos. Vamos usar agora o método `unique()`nesta coluna para ver quais são os valores únicos dessa coluna.

In [4]:
dados['BareNuc'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

3) Veja que os valores faltantes na coluna `BareNuc`são iguais a `'?'`. Use o método `replace()` em `df` para substituir `'?'` por `np.NaN`. Sobreescreva `df`para não perder a alteração realizada.

In [5]:
dados['BareNuc'] = dados['BareNuc'].replace('?',np.NaN)

4. Remova agora os dados faltantes usando `dropna()`

In [6]:
dados = dados.dropna()

Converta a coluna para `int` usando o método `astype()`.

In [7]:
dados['BareNuc'] = dados['BareNuc'].astype(int)

<ipython-input-7-9bb2c55b814c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['BareNuc'] = dados['BareNuc'].astype(int)


### Identificação das features e target

4) Crie um DataFrame `X` com todas as features. Lembrando que o target é o atributo Class e que ID não é uma feature.

In [8]:
x = dados.drop(['Class', 'ID'], axis=1)
x

,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1
...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1
695,2,1,1,1,2,1,1,1,1
696,5,10,10,3,7,3,8,10,2
697,4,8,6,4,3,4,10,6,1


5) Segmente o Dataframe para a representação do Target (Class)

In [9]:
y = dados['Class']
y

0      2
1      2
2      2
3      2
4      2
      ..
694    2
695    2
696    4
697    4
698    4
Name: Class, Length: 683, dtype: int64

## Padronização dos dados

A padronização de recursos é crucial para alguns algoritmos de aprendizado de máquina que consideram distâncias entre observações porque a distância entre duas observações difere para casos não padronizados e padronizados.

Maximizamos no SVM a distância até os pontos de dados mais próximos de diferentes classes. Portanto, a distância calculado entre os pontos de dados afeta o limite de decisão que o SVM escolhe. Em outras palavras, treinar um SVM sobre os dados padronizados e não padronizados leva à geração de modelos diferentes.

6. Use o `StandardScaler()` para padronizar os dados.

In [10]:
sc = StandardScaler()
x_padronizado = sc.fit_transform(x)
x_padronizado

array([[ 0.19790469, -0.70221201, -0.74177362, ..., -0.18182716,
        -0.61292736, -0.34839971],
       [ 0.19790469,  0.27725185,  0.26278299, ..., -0.18182716,
        -0.28510482, -0.34839971],
       [-0.51164337, -0.70221201, -0.74177362, ..., -0.18182716,
        -0.61292736, -0.34839971],
       ...,
       [ 0.19790469,  2.23617957,  2.2718962 , ...,  1.86073779,
         2.33747554,  0.22916583],
       [-0.15686934,  1.58320366,  0.93248739, ...,  2.67776377,
         1.02618536, -0.34839971],
       [-0.15686934,  1.58320366,  1.6021918 , ...,  2.67776377,
         0.37054027, -0.34839971]])

### Separação dos dados para teste e validação.

7) Divida os dados em treino e teste com a função `train_test_split()`. Separe 20% dos dados para validação.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

Verifique se as dimensões dos dados de teste e validação estão coerentes usando o método `shape()`.

In [12]:
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

X_train: (546, 9)
X_test: (137, 9)
y_train: (546,)
y_test: (137,)


### Treinamento e avaliação da SVM criada

O algoritmo SVM oferece uma escolha de funções de kernel para realizar seu processamento. Basicamente, o mapeamento de dados em um espaço dimensional superior é chamado de kernelling. A função matemática usada para a transformação é conhecida como função kernel e pode ser de diferentes tipos, como:

     1. Linear
     2. Polinômio
     3. Função de base radial (RBF)
     4. Sigmóide
Cada uma dessas funções tem suas características, seus prós e contras e sua equação, mas como não há uma maneira fácil de saber qual função tem o melhor desempenho com qualquer dataset, geralmente escolhemos funções diferentes e comparamos os resultados. 

8) Utilize a Função de base radial (RBF) como kernel. O classificador SVM é representado pelo método `SVC`

In [13]:
model = svm.SVC(kernel='rbf')
model.fit(X_train,y_train)
predict = model.predict(X_test)

9) Use o método `score()` para avaliar a performance de seu classificador.

In [14]:
model.score(X_test, y_test)

0.9708029197080292